# Как обрабатывал БД по недобросовестным поставщикам + как хотел применить классификацию для описания закупки, но не получилось

## Необходимые библиотеки

In [1]:
# для работы с БД
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# для работы с API openAI
import openai
# для визуального отслеживания прогресса длительных операций
from tqdm import tqdm

import xmltodict

## Обработаем БД: скорректируем ИНН, приведем цены к единой валюте, унифицируем причину добавления в РНП

In [2]:
# открываю таблицу с указанием типов данных
file_path = 'output_dataframe_cleaned_correct_datatype.csv'
dtype_dict = {'publish_date': 'string',
              'reason': 'string',
              'customer_code': 'string',
              'customer_inn': 'string',
              'customer_kpp': 'string',
              'supplier_full_name': 'string',
              'supplier_type': 'string',
              'supplier_inn': 'string',
              'supplier_kpp': 'string',
              'purchase_object': 'string',
              'contract_object': 'string',
              'contract_price_currency': 'string',
              'purchase_object': 'string'
             }  # словарь с типами данных для нужных столбцов
df = pd.read_csv(file_path, dtype=dtype_dict)

### Поработаем с ИНН

In [8]:
# Подсчитываем количество записей для каждой длины
# supplier_inn
count_per_length_supplier_inn = df['supplier_inn'].str.len().value_counts().sort_index()
count_per_length_supplier_inn

supplier_inn
8         2
9        15
10    49378
11       32
12    13197
13       14
14        2
15        4
16        1
20        1
Name: count, dtype: Int64

> У ИНН должно быть только 10 или 12 чисел (у ЮЛ и у ИП соотвественно)

> В нашем датасете есть неподходящие варианты. Это могут быть некорректные вводные данные, либо же это идентификаторы подрядчиков из других стран

> Работать с ними не получится (так как мэтчинг доступен только по рос компаниям), поэтому неподходящие ИНН удаляем

In [39]:
# Оставляем только те строки, длина 'supplier_inn' которых равна 10 или 12 символов
df = df.loc[df['supplier_inn'].astype(str).apply(len).isin([10, 12])]

### Поработаем с ценой контракта

> В данных присутствуют записи в других валютах. Переведем все в рубли по соответствующему курсу

USD

In [56]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2018) & (df['contract_price_currency'] == 'USD')
df.loc[condition, 'contract_price_amount'] *= 63 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

In [59]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2019) & (df['contract_price_currency'] == 'USD')
df.loc[condition, 'contract_price_amount'] *= 65 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

In [61]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2021) & (df['contract_price_currency'] == 'USD')
df.loc[condition, 'contract_price_amount'] *= 74 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

AUD

In [65]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2015) & (df['contract_price_currency'] == 'AUD')
df.loc[condition, 'contract_price_amount'] *= 46 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

In [67]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2016) & (df['contract_price_currency'] == 'AUD')
df.loc[condition, 'contract_price_amount'] *= 50 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

EUR

In [70]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2016) & (df['contract_price_currency'] == 'EUR')
df.loc[condition, 'contract_price_amount'] *= 74 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

In [72]:
# Изменение цены и валюты по условиям
condition = (df['publish_date_year'] == 2017) & (df['contract_price_currency'] == 'EUR')
df.loc[condition, 'contract_price_amount'] *= 66 # Изменяем цену
df.loc[condition, 'contract_price_currency'] = 'RUB'  # Изменяем валюту

### Поработаем с причиной добавления в РНП

In [ ]:
df['reason'].value_counts()

> Причина добавления в РНП содержит два основых значения: CANCEL_CONTRACT (причина на этапе исполнения контракта), WINNER_DEVIATION (причина на этапе заключения контракта)

> Остальные причины являются частью первых двух. По каким то причинам в некоторых записях их отмечали другими формулировками (или на русском). Отнесем их к своему типу.

In [ ]:
# Замена значений в столбце 'reason'
df['reason'] = df['reason'].replace({
    'ONE_WINNER_DEVIATION': 'WINNER_DEVIATION',
    'PARTICIPANT_DEVIATION_IF_WINNER_DEVIATION': 'WINNER_DEVIATION',
    'Уклонение победителя от заключения контракта': 'WINNER_DEVIATION',
    'Расторжение контракта': 'CANCEL_CONTRACT'
})

## Выбираем нужные переменные из обработанного датасета

> Для дальнейшей работы отберем необходимые столбцы из обработанного датасета

In [ ]:
needed_cols = df[['supplier_inn', 'publish_date_year', 'publish_date', 'reason', 'contract_price_amount']]
needed_cols.to_csv('rnp_suppliers.csv', index=False)

## Теперь как я пытался классифицировать описание закупки, но не получилось

> Переменные 'purchase_object', 'contract_object' содержут описание закупки, по типу "закупка по 3 видам медицинских шприцов" и т.д

> Каждое описание - уникальное и не имеет конкретной структуры - видимо это заполняется человеком

> Коротко об идее, что я хотел сделать и для чего:
> 1) классифицировать описание закупки по ОКВЭД с помощью LLM
> 2) сравнивать данный ОКВЭД с ОКВЭДом, по которому работает данный поставщик
> 3) получить ответ на вопрос: а выполняет ли поставщик закупку в соотвествии со своим видом деятельности или нет?

In [28]:
# Проверка заполненности столбцов
total_entries = len(df)
non_null_purchase = df['purchase_object'].notna().sum()
non_null_contract = df['contract_object'].notna().sum()

print(f"Заполненность столбца purchase_object: {non_null_purchase} из {total_entries}")
print(f"Заполненность столбца contract_object: {non_null_contract} из {total_entries}")

# Подсчет случаев, когда X заполнен, а Y нет, и наоборот
purchase_filled_contract_empty = ((df['purchase_object'].notna()) & (df['contract_object'].isna())).sum()
contract_filled_purchase_empty = ((df['contract_object'].notna()) & (df['purchase_object'].isna())).sum()

print(f"Случаев, когда purchase заполнен, а contract нет: {purchase_filled_contract_empty}")
print(f"Случаев, когда contract заполнен, а purchase нет: {contract_filled_purchase_empty}")

Заполненность столбца purchase_object: 58274 из 62575
Заполненность столбца contract_object: 55127 из 62575
Случаев, когда purchase заполнен, а contract нет: 6563
Случаев, когда contract заполнен, а purchase нет: 3416


In [29]:
df['final_object'] = df['contract_object'].combine_first(df['purchase_object'])

In [37]:
df['final_object'].isna().sum()

885

> тут я использую "gpt-3.5-turbo-instruct"

> было проведено много разных итераций и промтов, в итоге остановился на этом

> но это выходило в немалую денежку, и проводить это на всем датасете я не хотел

> также на тестовом примере (в 2000 записей) было обнаружено, что в некоторых моментах модель ошибается и присваивает неверный ОКВЭД

In [ ]:
tqdm.pandas()

openai.api_key = '...' # индивидуальный ключ для API

def classify_purchase(description):
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",  
            prompt=f"К какому виду деятельности относится следующее описание закупки? '{description}'
                \n\n1) Сельское, лесное хозяйство, охота, рыболовство и рыбоводство\n
                    2) Добыча полезных ископаемых\n
                    3) Обрабатывающие производства\n
                    4) Обеспечение электрической энергией, газом и паром; кондиционирование воздуха\n
                    5) Водоснабжение; водоотведение, организация сбора и утилизации отходов, деятельность по ликвидации загрязнений\n
                    6) Строительство\n
                    7) Торговля оптовая и розничная; ремонт автотранспортных средств и мотоциклов\n
                    8) Транспортировка и хранение\n
                    9) Деятельность гостиниц и предприятий общественного питания\n
                    10) Деятельность в области информации и связи\n
                    11) Деятельность финансовая и страховая\n
                    12) Деятельность по операциям с недвижимым имуществом\n
                    13) Деятельность профессиональная, научная и техническая\n
                    14) Деятельность административная и сопутствующие дополнительные услуги\n
                    15) Государственное управление и обеспечение военной безопасности; социальное обеспечение\n
                    16) Образование\n
                    17) Деятельность в области здравоохранения и социальных услуг\n
                    18) Деятельность в области культуры, спорта, организации досуга и развлечений\n
                    19) Предоставление прочих видов услуг\n
                    20) Деятельность домашних хозяйств как работодателей; недифференцированная деятельность частных домашних хозяйств по производству товаров и оказанию услуг для собственного потребления\n
                    21) Деятельность экстерриториальных организаций и органов\n Если нет точной информации, выведи 'Нет ответа'",
            temperature=0,
            max_tokens=100
        )
        result = response.choices[0].text.strip()
        return result.split(') ')[1] if ') ' in result else result
    except Exception as e:
        print(f"Error: {e}")
        return "Classification Error"

df['clean_object'] = df['final_object'].progress_apply(classify_purchase)